# Time Series Analysis

## Predicting Future Values
First, we will show that $\hat{u}(t+m)$ is the minimizer of:
$$ \mathbb{E}[ \vert u(t+m) - \hat{u}(t+m) \vert^2 ] $$

In other words:
$$ \hat{u}(t+m) = \mathbb{E}[ u(t+m)\ \vert\ u(t-n), \ldots, u(t-1) ] $$

**INCOMPLETE**

## Stationary Stochastic Processes

We will compute the covariance matrices of the following stochastic system:
$$ du = -udt + dw $$

We will evaluate this system using the Euler-Marayama scheme:

In [1]:
from numpy.random import normal
from typing import Callable

def euler_murayama_update (
    position,           # u, float
    forcing_term,       # a, callable
    timestep,           # n, int
    timescale,          # dt, float
    chaos=1.            # used to dampen or strengthen Brownian term
):
    return position + forcing_term(timestep * timescale, position) * timescale + chaos * normal(loc=0., scale=timescale)